In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
import requests as rq

In [2]:
spark = SparkSession.builder.master("spark://spark-master:7077").getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/08 12:36:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
df = spark.read.options(header=True).options(inferSchema='True').csv("/opt/workspace/spark-homework/raw-data")
df.printSchema()

root
 |-- Id: long (nullable = true)
 |-- Name: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)



In [6]:
df = df.filter('Latitude is Not NULL').filter('Longitude is Not NULL')
df = df[~df.col1.isnan() & !]

In [7]:
df.show()

+------------+--------------------+-------+---------------+--------------------+----------+------------+
|          Id|                Name|Country|           City|             Address|  Latitude|   Longitude|
+------------+--------------------+-------+---------------+--------------------+----------+------------+
|           2|Parkside Inn At I...|     US|Incline Village|1003 Tahoe Boulev...| 39.244493| -119.936437|
|  8589934592|      Cadillac Motel|     US|     Brandywine|     16101 Crain Hwy|  38.66893|   -76.87629|
| 17179869184|  Days Inn Brookings|     US|      Brookings|         2500 6th St|  44.31141|   -96.76286|
| 17179869187|             Motel 6|     US|       Grayling|     6843 W M 72 Hwy| 44.657326|   -84.74439|
| 25769803780|      Carleton Hotel|     US|       Carleton|       927 Monroe St|  42.05927|   -83.39095|
| 42949672965|Comfort Inn Delan...|     US|         Deland|400 E Internation...| 29.054737|  -81.297208|
| 51539607555|Magnuson Hotel Su...|     US|      Summer

In [9]:
@udf
def forward_eval(lat, lon, name, city):
    response = rq.get(f'https://api.opencagedata.com/geocode/v1/json?q={lat}+{lon}&key=bd6356b6fa7446e89545e28dc1bd8bab')
    if response.status_code != 200:
        return
    
    response_data = response.json()['results'][0]['components']

    building_true = response_data['building']
    city_true = response_data['city']

    if name in building_true or building_true in name and city_true in city or city in city_true:
        return True
    else:
        return False
    

In [10]:
df = df.withColumn('geo_filter', forward_eval(df.Latitude, df.Longitude, df.Name, df.City))

In [11]:
df.show()

22/02/08 12:38:38 WARN TaskSetManager: Lost task 0.0 in stage 3.0 (TID 5) (172.18.0.5 executor 1): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/bin/spark-3.2.0-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/worker.py", line 603, in main
    func, profiler, deserializer, serializer = read_udfs(pickleSer, infile, eval_type)
  File "/usr/bin/spark-3.2.0-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/worker.py", line 449, in read_udfs
    udfs.append(read_single_udf(pickleSer, infile, eval_type, runner_conf, udf_index=i))
  File "/usr/bin/spark-3.2.0-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/worker.py", line 251, in read_single_udf
    f, return_type = read_command(pickleSer, infile)
  File "/usr/bin/spark-3.2.0-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/worker.py", line 71, in read_command
    command = serializer._read_with_length(file)
  File "/usr/bin/spark-3.2.0-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/serializers.py", line 160, in 

PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "/usr/bin/spark-3.2.0-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/worker.py", line 603, in main
    func, profiler, deserializer, serializer = read_udfs(pickleSer, infile, eval_type)
  File "/usr/bin/spark-3.2.0-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/worker.py", line 449, in read_udfs
    udfs.append(read_single_udf(pickleSer, infile, eval_type, runner_conf, udf_index=i))
  File "/usr/bin/spark-3.2.0-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/worker.py", line 251, in read_single_udf
    f, return_type = read_command(pickleSer, infile)
  File "/usr/bin/spark-3.2.0-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/worker.py", line 71, in read_command
    command = serializer._read_with_length(file)
  File "/usr/bin/spark-3.2.0-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/serializers.py", line 160, in _read_with_length
    return self.loads(ob